<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/Mask2Former/Inference_with_Mask2Former.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inference with Mask2Former

[Mask2Former](https://huggingface.co/docs/transformers/main/en/model_doc/mask2former) is a very nice new model from Meta AI, capable of solving any type of image segmentation (whether it's instance, semantic or panoptic segmentation) using the same architecture. The model improves upon [DETR](https://huggingface.co/docs/transformers/model_doc/detr) and [MaskFormer](https://huggingface.co/docs/transformers/model_doc/maskformer) by incorporating masked attention in its Transformer decoder.

<img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/model_doc/mask2former_architecture.jpg" alt="drawing" width="500"/>

In this notebook, we'll illustrate inference with this model (i.e. making predictions on a new image) for panoptic, semantic and instance segmentation.

## Set-up environment

First, we install 🤗 Transformers. We install from Github here since the Mask2Former model is brand new at the time of writing.

In [3]:
%pip install -q git+https://github.com/huggingface/transformers.git

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Note: you may need to restart the kernel to use updated packages.


## Define model

Next, we instantiate a Mask2Former model from the [hub](https://huggingface.co/models?other=mask2former), along with its image processor.

Note that the authors open-sourced many checkpoints, trained on instance, semantic and panoptic segmentation datasets. Here we load a Mask2Former checkpoint trained on the COCO-panoptic dataset.

In [2]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-base-coco-panoptic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-base-coco-panoptic")

ModuleNotFoundError: No module named 'transformers'

## Load image

I usually run inference on the cats image. It's part of the COCO dataset 🐈.

In [ ]:
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
image

We prepare the image for the model using the image processor.

In [ ]:
inputs = processor(images=image, return_tensors="pt")
for k,v in inputs.items():
  print(k,v.shape)

In [ ]:
inputs

## Forward pass

Next, we can forward the `pixel_values` and `pixel_mask` through the model. We use the `torch.no_grad()` context manager as we don't need gradients to be computed (this is only useful when training a model). This will save us a lot of memory.

In [ ]:
import torch

with torch.no_grad():
  outputs = model(**inputs)

## Visualize

For visualization, we can postprocess the outputs of the model. In case of panoptic segmentation, the model predicts 2 things per image: 1) a segmentation map 2) a corresponding segments_info (which contains more information regarding the segments in the segmentation map).

In [ ]:
# you can pass them to processor for postprocessing
results = processor.post_process_panoptic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
print(results.keys())

In [ ]:
for segment in results['segments_info']:
  print(segment)

In this case, the model has recognized 5 different segments in the image.

In [ ]:
segment_to_label = {segment['id']: segment['label_id'] for segment in results["segments_info"]}
print(segment_to_label)

Let's visualize the binary mask of the first segment:

In [ ]:
import numpy as np

def get_mask(segment_id):
  print("Visualizing mask for:", model.config.id2label[segment_to_label[segment_id]])

  mask = (results['segmentation'].numpy() == segment_id)
  visual_mask = (mask * 255).astype(np.uint8)
  visual_mask = Image.fromarray(visual_mask)

  return visual_mask

# note: segment with id == 0 means "background",
# so we visualize segment with id == 1 here
get_mask(segment_id=1)

We can create a single visualization for all segments:

In [ ]:
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm

def draw_panoptic_segmentation(segmentation, segments_info):
    # get the used color map
    viridis = cm.get_cmap('viridis', torch.max(segmentation))
    fig, ax = plt.subplots()
    ax.imshow(segmentation)
    instances_counter = defaultdict(int)
    handles = []
    # for each segment, draw its legend
    for segment in segments_info:
        segment_id = segment['id']
        segment_label_id = segment['label_id']
        segment_label = model.config.id2label[segment_label_id]
        label = f"{segment_label}-{instances_counter[segment_label_id]}"
        instances_counter[segment_label_id] += 1
        color = viridis(segment_id)
        handles.append(mpatches.Patch(color=color, label=label))
        
    ax.legend(handles=handles)

draw_panoptic_segmentation(**results)

Here, we can see that the model is capable of detecting the individual cats and remotes in the image. Semantic segmentation on the other hand would just create a single mask for the "cat" category.

## Inference (semantic segmentation)

We can do the same as above, but with a checkpoint fine-tuned on a semantic segmentation dataset (in this case, CityScapes).

In [ ]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-small-cityscapes-semantic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-small-cityscapes-semantic")

Let's load an image from the CityScapes dataset.

In [ ]:
from PIL import Image
import requests

url = 'https://cdn-media.huggingface.co/Inference-API/Sample-results-on-the-Cityscapes-dataset-The-above-images-show-how-our-method-can-handle.png'
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
inputs = processor(images=image, return_tensors="pt")
for k,v in inputs.items():
  print(k,v.shape)

In [ ]:
import torch

# forward pass
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
# you can pass them to processor for postprocessing
predicted_map = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
print(predicted_map.shape)

We can draw the semantic segmentation map (with one label per pixel) over the image:

In [ ]:
# generate random color palette, which maps each class to a RGB value
color_palette = [list(np.random.choice(range(256), size=3)) for _ in range(len(model.config.id2label))]
print(color_palette)

In [ ]:
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

seg = predicted_map
color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3
palette = np.array(color_palette)
for label, color in enumerate(palette):
    color_seg[seg == label, :] = color
# Convert to BGR
color_seg = color_seg[..., ::-1]

# Show image + mask
img = np.array(image) * 0.5 + color_seg * 0.5
img = img.astype(np.uint8)

plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.show()

## Inference (instance segmentation)

Inference for instance segmentation is very similar.

In [ ]:
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-coco-instance")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-coco-instance")

In [ ]:
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)
image

In [ ]:
inputs = processor(images=image, return_tensors="pt")
for k,v in inputs.items():
  print(k,v.shape)

In [ ]:
import torch

# forward pass
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
outputs.keys()

In [ ]:
torch.where(outputs.class_queries_logits.argmax(-1) == 65)

In [ ]:
outputs.masks_queries_logits[:,160,:,:]

In [ ]:
# you can pass them to processor for postprocessing
results = processor.post_process_instance_segmentation(outputs, target_sizes=[image.size[::-1]], threshold=0.9)[0]
print(results.keys())

In [ ]:
results['segmentation']

In [ ]:
for segment in results['segments_info']:
  print(segment)

In [ ]:
segment_to_label = {segment['id']: segment['label_id'] for segment in results["segments_info"]}
print(segment_to_label)

In [ ]:
import numpy as np

def get_mask(segment_id):
  print("Visualizing mask for:", model.config.id2label[segment_to_label[segment_id]])

  mask = (results['segmentation'].numpy() == segment_id)
  visual_mask = (mask * 255).astype(np.uint8)
  visual_mask = Image.fromarray(visual_mask)

  return visual_mask

# note: segment with id == 0 means "background",
# so we visualize segment with id == 1 here
get_mask(segment_id=1)

In [ ]:
get_mask(segment_id=2)

In [ ]:
get_mask(segment_id=3)

In [ ]:
get_mask(segment_id=4)